In [25]:
from glob import glob
from time import time
from global_config import FOLDER_IMAGES

In [26]:
list_images = glob(FOLDER_IMAGES + "*")
n_images = len(list_images)

print(n_images)

5763


In [27]:
def load_graph_from_pb(pb_file):
    graph = tf.Graph()

    graph_def = tf.GraphDef()
    graph_def.ParseFromString(open(pb_file, 'rb').read())

    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph

def load_graph_from_meta(model_dir, meta_file, checkpoint_prefix=None):
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph(model_dir+meta_file)
        if checkpoint_prefix is None:
            saver.restore(sess, tf.train.latest_checkpoint(model_dir))
        else:
            saver.restore(sess, model_dir + checkpoint_prefix)
        graph = tf.get_default_graph()
    return graph

def save_graph(graph, output_file, as_pbtxt=False):
    folder, file = split(output_file)
    tf.train.write_graph(graph, logdir=folder, name=file, as_text=as_pbtxt)
    
def get_graph_node_names(graph):
    return [node.name for node in graph.as_graph_def().node]

# TropComplique

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw

In [4]:
MODEL_PATH = 'C:/Users/arnal/Downloads/face_detectors/trop_complique.pb'

In [ ]:
class FaceDetector:
    def __init__(self, model_path, gpu_memory_fraction=0.25, visible_device_list='0'):
        """
        Arguments:
            model_path: a string, path to a pb file.
            gpu_memory_fraction: a float number.
            visible_device_list: a string.
        """
        with tf.gfile.GFile(model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        graph = tf.Graph()
        with graph.as_default():
            tf.import_graph_def(graph_def, name='import')

        self.input_image = graph.get_tensor_by_name('import/image_tensor:0')
        self.output_ops = [
            graph.get_tensor_by_name('import/boxes:0'),
            graph.get_tensor_by_name('import/scores:0'),
            graph.get_tensor_by_name('import/num_boxes:0'),
        ]

        gpu_options = tf.GPUOptions(
            per_process_gpu_memory_fraction=gpu_memory_fraction,
            visible_device_list=visible_device_list
        )
        config_proto = tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False)
        self.sess = tf.Session(graph=graph, config=config_proto)

    def __call__(self, image, score_threshold=0.5):
        """Detect faces.

        Arguments:
            image: a numpy uint8 array with shape [height, width, 3],
                that represents a RGB image.
            score_threshold: a float number.
        Returns:
            boxes: a float numpy array of shape [num_faces, 4].
            scores: a float numpy array of shape [num_faces].

        Note that box coordinates are in the order: ymin, xmin, ymax, xmax!
        """
        h, w, _ = image.shape
        image = np.expand_dims(image, 0)

        boxes, scores, num_boxes = self.sess.run(
            self.output_ops, feed_dict={self.input_image: image}
        )
        num_boxes = num_boxes[0]
        boxes = boxes[0][:num_boxes]
        scores = scores[0][:num_boxes]

        to_keep = scores > score_threshold
        boxes = boxes[to_keep]
        scores = scores[to_keep]

        scaler = np.array([h, w, h, w], dtype='float32')
        boxes = boxes * scaler

        return boxes, scores

In [12]:
def load_image(path):
    image_array = cv2.imread(path)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    return image_array

def draw_boxes_on_image(image, boxes, scores):
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image.size

    for b, s in zip(boxes, scores):
        ymin, xmin, ymax, xmax = b
        fill = (255, 0, 0, 45)
        outline = 'red'
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
        draw.text((xmin, ymin), text='{:.3f}'.format(s))
    return image_copy

In [30]:
n_detected = 0

face_detector = FaceDetector(MODEL_PATH, gpu_memory_fraction=0.0, visible_device_list='0')

start = time()
for i, image_path in enumerate(list_images, start=1):
    print(f'{i} of {n_images}', end='\r')
    
    im = load_image(image_path)
    boxes, scores = face_detector(im, score_threshold=0.3)
    if len(boxes) > 0:
        n_detected += 1
end = time()

avg_time = (end - start) / n_images

print(f'Total Detections: {n_detected} of {n_images}')
print('Avg Time (s):', avg_time)

Total Detections: 5746 of 5763
Avg Time (s): 0.11038639333209664


In [23]:
from os.path import split

graph = load_graph_from_meta("C:/Users/arnal/Downloads/face_detectors/run00/", "model.ckpt-240000.meta")
save_graph(graph, "C:/Users/arnal/Downloads/face_detectors/run00/frozen_graph.pb")
save_graph(graph, "C:/Users/arnal/Downloads/face_detectors/run00/frozen_graph.pbtxt", as_pbtxt=True)

INFO:tensorflow:Restoring parameters from C:/Users/arnal/Downloads/face_detectors/run00/model.ckpt-240000


In [22]:
graph = load_graph_from_pb(MODEL_PATH)
# graph = load_graph_from_meta("C:/Users/arnal/Downloads/face_detectors/run00/", "model.ckpt-240000.meta")
name_list = get_graph_node_names(graph)

with open('name-list.txt', 'w') as file:
    for name in name_list:
        file.write(name+'\n')

# Yeephycho 

In [3]:
import cv2
import numpy as np
import tensorflow as tf

In [4]:
MODEL_PATH = 'C:/Users/arnal/Downloads/face_detectors/yeephycho.pb'

In [5]:
def load_graph_from_pb(pb_file):
    graph = tf.Graph()

    graph_def = tf.GraphDef()
    graph_def.ParseFromString(open(pb_file, 'rb').read())

    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph

def run_graph(graph, inputs_list, input_tensor_names, output_tensor_names):
    assert(len(inputs_list) == len(input_tensor_names))

    with tf.Session(graph=graph) as sess:
        input_tensors = [graph.get_tensor_by_name(name) for name in input_tensor_names]
        output_tensors = [graph.get_tensor_by_name(name) for name in output_tensor_names]
        feed_dict = {tensor:value for tensor, value in zip(input_tensors, inputs_list)}
        return sess.run(output_tensors, feed_dict=feed_dict)

class Rect():
    def __init__(self, x, y, width=None, height=None):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def from_tl_br_points(tl, br):
        return Rect(tl[0], tl[1], br[0] - tl[0], br[1] - tl[1])

    def tl(self):
        '''return upper left point (x, y)'''
        return (self.x, self.y)

    def br(self):
        '''return bottom right point (x, y)'''
        return (self.x + self.width, self.y + self.height)

    def __str__(self):
        return '(x, y, width, height): ({}, {}, {}, {})'.format(self.x, self.y, self.width, self.height)

class Yeephycho():
    def __init__(self, conf_threshold=0.1):
        self.conf_threshold = conf_threshold
        self.graph = load_graph_from_pb(MODEL_PATH)

    def detect(self, image_file, det_size=(512, 512)):
        det_size = (512, 512)
        im = cv2.imread(image_file, cv2.IMREAD_COLOR)
        h, w = im.shape[:2]
#         im = cv2.resize(im, det_size)

        inputs_list = [np.expand_dims(im, axis=0)]
        input_tensor_names = ['import/image_tensor:0']
        output_tensor_names = ['import/detection_boxes:0', 'import/detection_scores:0']
        boxes, scores = run_graph(self.graph, inputs_list, input_tensor_names, output_tensor_names)

        boxes = boxes[scores > self.conf_threshold]
        return [
            Rect.from_tl_br_points(
                (int(x1 * w), int(y1 * h)),
                (int(x2 * w), int(y2 * h))
            ) 
            for y1, x1, y2, x2 in boxes
        ]

In [6]:
n_detected = 0

face_detector = Yeephycho()

start = time()
for i, image_path in enumerate(list_images, start=1):
    print(f'{i} of {n_images}', end='\r')
    
    rects = face_detector.detect(image_path)
    if len(rects) > 0:
        n_detected += 1
end = time()

avg_time = (end - start) / n_images

print(f'Total Detections: {n_detected} of {n_images}')
print('Avg Time (s):', avg_time)

Total Detections: 5705 of 5763
Avg Time (s): 1.8717303832909846


# Faced

In [24]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
MODEL_PATH = 'C:/Users/arnal/Downloads/face_detectors/faced.pb'